# RG374 LegendPlex analysis IFNAR fl LysMcre CpG

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(

            library(outliers), # Grubbs outlier detection 
            
            # Data 
            library(tidyverse), 
            library(data.table), 
            library(reactable), 

            # Plotting 
            library(ComplexHeatmap), 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel)

        )
    )
)

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Parameter 

In [ ]:
color$sample_group <- c("D0 +/+"="#66c2a5", "D0 cre/+"="#00634A", "D3 +/+"="#cd34b5", "D3 cre/+"="#FFAC1E", "D6 +/+"="#cd34b5", "D6 cre/+"="#FFAC1E")

In [ ]:
contrast_1 <- c(
    
    "(D0 +/+) - (D3 +/+)", 
    "(D0 +/+) - (D6 +/+)", 
    
    "(D0 cre/+) - (D3 cre/+)", 
    "(D0 cre/+) - (D6 cre/+)", 
    
    "(D0 +/+) - (D0 cre/+)", 
    "(D3 +/+) - (D3 cre/+)", 
    "(D6 +/+) - (D6 cre/+)"
    
)

# Helper function

In [ ]:
sidak <- function(mat, formula_aov, formula_sidak, verbose=FALSE) {
    
    # Perform two-way ANOVA
    anova_result <- stats::aov(formula_aov, data=mat)

    # Post-hoc pairwise comparisons with Sidak correction
    pairwise_comparisons <- emmeans::emmeans(anova_result, formula_sidak, adjust="sidak")
    
    # View the results
    if(verbose) {

        print(pairwise_comparisons$emmeans)  # Estimated marginal means
        print(pairwise_comparisons$contrasts)  # Pairwise comparisons with Sidak adjustment

    }

    # Optional: Display pairwise comparisons in a readable format
    res <- summary(pairwise_comparisons$contrasts)
    
    return(res)
    
}

In [ ]:
replace_greek_abg <- function(x) {
    
    map <- c(
        
        "\u03B1"="a", "\u0391"="A",
        "\u03B2"="b",  "\u0392"="B",
        "\u03B3"="g", "\u0393"="G"
    
    )
    
    f <- function(s) {
        
        s <- enc2utf8(s)
        for (k in names(map)) s <- gsub(k, map[[k]], s, fixed=TRUE)
        return(s)
    
    }
    
    if (is.factor(x)) {
        
        lev <- vapply(levels(x), f, "", USE.NAMES=FALSE)
        levels(x) <- lev
        return(x)
        
    } else if (is.list(x)) {
        
        lapply(x, replace_greek_abg)
        
    } else {
        
        vapply(x, f, "", USE.NAMES=FALSE)
    
    }

}

In [ ]:
bar_plot <- function(mat, x, y, xlab="", ylab="", ggtitle="", scale_y=FALSE, parse_y=FALSE, expand_y=NULL, position_bar="stack", position_jitter="jitter", position="default") {

    ggtitle <- replace_greek_abg(ggtitle)
    
    if(scale_y) {
        
        res <- scale_label(mat, deparse(substitute(y)))
        mat <- res[[1]]
        ylab <- res[[2]]

    }

    if(position=="dodge") {

        position_bar <- position_dodge(width=0.8)
        position_jitter <- position_dodge(width=0.8)
    }
    
    p <- ggplot(mat, aes(x={{x}}, y={{y}}, color=sample_group, fill=sample_group)) + 
    
        geom_bar(stat="summary", width=0.8, color="black", size=0.1, fun=mean, position=position_bar) + 
        
        stat_summary(fun.data=function(x) {
            m <- mean(x)
            sem <- sd(x, na.rm=TRUE) / sqrt(sum(!is.na(x)))
            ymin <- mean(x)
            ymax <- m + sem
            data.frame(y=m, ymin=ymin, ymax=ymax)
        }, geom="errorbar", width=0.25, color="black") +
    
        geom_jitter(shape=21, stroke=0.1, size=1.0, show.legend=FALSE, color="black", position=position_jitter) + 
        xlab(xlab) + ylab(ylab) + ggtitle(ggtitle) + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group) + 
        theme(axis.text.x=element_text(angle=90, vjust=0.5, hjust=1)) + 
        guides(fill=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0.25, keyheight=0.25, default.unit="cm"))
    
    if(parse_y) {
        
        p <- p + ylab(parse(text=ylab))
        
    }


    if(!is.null(expand_y)) {

         p <- p + ylim(0, (max(ggplot_build(p)$data[[1]]$y) + expand_y))
    }
    
    return(p)
    
}

# Plasma 

In [ ]:
mat <- read.csv("data/RG374_ifnar_fl_lysmcre_cpg/legendplex/plasma/RG374_ifnar_fl_lysmcre_cpg_plasma_legendplex_data_2024_08_23.csv") %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    rename_with(~gsub(".", "-", ., fixed=TRUE)) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cytokine", value.name=c("concentration")) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [ ]:
mat <- split(mat, f=paste(mat$sample_group, mat$cytokine))

In [ ]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$concentration))>=3) {
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(concentration)$p.value, 
            grubbs_alt=grubbs.test(concentration)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & concentration == max(concentration) ~ TRUE,
                # p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & concentration == min(concentration) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [ ]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)

In [ ]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [ ]:
# Sidak test
stat <- lapply(levels(mat$cytokine), function(i) {sidak(mat[mat$cytokine==i, ], formula(concentration ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE) %>% dplyr::mutate(cytokine=i)})
stat <- do.call("rbind", stat)

In [ ]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$cytokine), function(x) bar_plot(x, sample_group, concentration, ylab="pg/ml", ggtitle=x$cytokine[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [ ]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_2/bp_legendplex_plasma.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()

# Spleen 

In [ ]:
mat <- read.csv("data/RG374_ifnar_fl_lysmcre_cpg/legendplex/spleen/RG374_ifnar_fl_lysmcre_cpg_spleen_legendplex_data_2024_08_23.csv") %>% 
    dplyr::select(-Mutation.1, -Grad.1, -Mutation.2, -sample) %>% 
    dplyr::rename(genotype=Grad.2, time_point=well) %>% 
    rename_with(~gsub(".", "-", ., fixed=TRUE)) %>% 
    reshape2::melt(., id.vars=c("genotype", "time_point"), variable="cytokine", value.name=c("concentration")) %>%
    dplyr::mutate(genotype=factor(genotype, levels=c("+/+", "cre/+"))) %>% 
    dplyr::mutate(time_point=ifelse(time_point=="baseline", "D0", ifelse(time_point=="day3", "D3", "D6"))) %>% 
    dplyr::mutate(sample_group=paste(time_point, genotype)) %>% 
    dplyr::mutate(sample_group=factor(sample_group, levels=c("D0 +/+", "D0 cre/+", "D3 +/+", "D3 cre/+", "D6 +/+", "D6 cre/+"))) %>% 
    dplyr::mutate(time_point=factor(time_point, levels=c("D0", "D3", "D6")))

In [ ]:
mat <- split(mat, f=paste(mat$sample_group, mat$cytokine))

In [ ]:
mat <- lapply(mat, function(x) {

    if(length(unique(x$concentration))>=3) {
        x <- x %>% dplyr::mutate(
            
            p_value_grubbs=grubbs.test(concentration)$p.value, 
            grubbs_alt=grubbs.test(concentration)$alternative,
            grubbs_res=case_when(
                
                p_value_grubbs < 0.05 & grepl("highest", grubbs_alt) & concentration == max(concentration) ~ TRUE,
                # p_value_grubbs < 0.05 & grepl("lowest",  grubbs_alt) & concentration == min(concentration) ~ TRUE,
                TRUE ~ FALSE
            
            )
        )
    } else {

        x <- x %>% dplyr::mutate(

            p_value_grubbs=NA, 
            grubbs_alt=NA,
            grubbs_res=FALSE
            
        )
        
    }

    return(x)
}
             )

In [ ]:
mat <- do.call("rbind", mat)
table(mat$grubbs_res)

In [ ]:
# Remove outlier
mat <- mat[!mat$grubbs_res, ]

In [ ]:
# Sidak test
stat <- lapply(levels(mat$cytokine), function(i) {sidak(mat[mat$cytokine==i, ], formula(concentration ~ time_point * genotype), formula(pairwise ~ time_point * genotype), verbose=FALSE) %>% dplyr::mutate(cytokine=i)})
stat <- do.call("rbind", stat)

In [ ]:
stat[stat$contrast %in% contrast_1 & stat$p.value<=0.05, ] %>% dplyr::mutate(p.value=round(p.value, digits=3))

In [ ]:
bar_plot_1 <- lapply(split(mat, f=mat$cytokine), function(x) bar_plot(x, sample_group, concentration, ylab="pg/ml", ggtitle=x$cytokine[1]) + theme(legend.position="none") + theme_global_set(4))
bar_plot_1 <- lapply(bar_plot_1, function(p) egg::set_panel_size(p, width=unit(1.8, "cm"), height=unit(2.0, "cm")))
bar_plot_1 <- do.call(gridExtra::arrangeGrob, c(bar_plot_1, ncol=7, nrow=ceiling(length(bar_plot_1)/7)))

In [ ]:
pdf("result/figures/3_RG374_ifnar_fl_lysmcre_cpg/panel_2/bp_legendplex_spleen.pdf", width=7*1.5, height=1.9*ceiling(length(bar_plot_1)/7))

grid::grid.draw(bar_plot_1)

dev.off()